In [1]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.tree import DecisionTreeClassifier

In [2]:
df = pd.read_csv("train.csv")

# 1. Without Pipeline

In [3]:
df.head(3)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S


In [4]:
df.drop(["Name", "PassengerId","Ticket","Cabin"],inplace =True,axis =1)

In [5]:
#Step 1: Train-test-split
X_train,X_test,y_train,y_test = train_test_split(df.drop(columns = ["Survived"]), df["Survived"],test_size = 0.2,random_state = 42)

In [6]:
df.isnull().sum()

Survived      0
Pclass        0
Sex           0
Age         177
SibSp         0
Parch         0
Fare          0
Embarked      2
dtype: int64

In [7]:
# Applying Imputation

# SimpleImputer
si_age = SimpleImputer()
si_embarked = SimpleImputer(strategy = "most_frequent")

X_train_age = si_age.fit_transform(X_train[["Age"]])
X_train_embarked = si_embarked.fit_transform(X_train[["Embarked"]])

X_test_age = si_age.transform(X_test[["Age"]])
X_test_embarked = si_embarked.transform(X_test[["Embarked"]])




In [8]:
# ONe HOt Encoding on Sex and Emabarked

ohe_sex = OneHotEncoder(sparse_output =False, handle_unknown= "ignore")
ohe_embarked = OneHotEncoder(sparse_output =False, handle_unknown= "ignore")

X_train_sex = ohe_sex.fit_transform(X_train[["Sex"]])
X_train_embarked = ohe_embarked.fit_transform(X_train_embarked)

X_test_sex = ohe_sex.transform(X_test[["Sex"]])
X_test_embarked = ohe_embarked.transform(X_test_embarked)





In [9]:
X_test[["Sex"]].value_counts()

Sex   
male      110
female     69
Name: count, dtype: int64

In [10]:
X_train_rem = X_train.drop(columns = ["Sex","Age","Embarked"])
X_test_rem = X_test.drop(columns = ["Sex","Age","Embarked"])


In [11]:
X_train_trf = np.concatenate((X_train_rem,X_train_age,X_train_sex, X_train_embarked),axis = 1)
X_test_trf = np.concatenate((X_test_rem,X_test_age,X_test_sex, X_test_embarked),axis = 1)


In [15]:
X_train_trf.shape, X_test_trf.shape

((712, 10), (179, 10))

In [16]:
clf = DecisionTreeClassifier()
clf.fit(X_train_trf, y_train)

DecisionTreeClassifier()

In [17]:
y_pred = clf.predict(X_test_trf)

In [18]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.7821229050279329

In [19]:
import pickle

In [22]:
pickle.dump(ohe_sex, open("models/ohe_sex.pkl", "wb"))
pickle.dump(ohe_embarked, open("models/ohe_embarked.pkl","wb"))
pickle.dump(clf, open("models/clf.pkl","wb"))